# Openslide


## Installation ([link](https://openslide.org/api/python/#installing))

-   Linux or MacOS

    1. **Openslide** 설치: Package Manager를 이용하여 설치함
    2. **Openslide Python** 설치: Package Manager 혹은 PIP를 이용하여 설치

    -   1, 2 모두 Package Manager를 이용하는 경우, 동일한 Package Manager를 사용할 것

-   Windows

    1. Windows Binaries를 다운받은 후 원하는 경로에 압축해제
    2. 직접 경로를 지정하여 사용함

    -   `with os.add_dll_directory(r'c:\path\to\openslide-win64\bin')` 내부에서 openslide를 import 함


In [ ]:
import os

SLIDE_FOLDER_PATH = "slides"
SLIDE_FILE_PATH = os.path.join(SLIDE_FOLDER_PATH, os.listdir(SLIDE_FOLDER_PATH)[0])

## Openslide Object

-   원본 이미지 1장 -> downscale 하여 작은 이미지 생성한 것


In [ ]:
from openslide import OpenSlide

print(OpenSlide.detect_format(SLIDE_FILE_PATH))  # classmethod

In [ ]:
with OpenSlide(SLIDE_FILE_PATH) as slide:
    for name, value in slide.properties.items():
        print(f"{name:<40} {value}")

In [ ]:
with OpenSlide(SLIDE_FILE_PATH) as slide:
    print("level_count:", slide.level_count)

    print("level_dimensions:")
    for i, value in enumerate(slide.level_dimensions):
        print(f"\t{i:<2} {value}")

    print("level_downsamples:")
    for i, value in enumerate(slide.level_downsamples):
        print(f"\t{i:<2} {value}")

    print("best level for downsampling by a factor of 10:", slide.get_best_level_for_downsample(10))

In [ ]:
with OpenSlide(SLIDE_FILE_PATH) as slide:
    print("associated_images:")
    for name, image in slide.associated_images.items():
        print(f"\t{name}:", image)
        width, height = image.size
        ratio = width / height
        new_height = 200
        image.resize((int(new_height * ratio), new_height)).show()

In [ ]:
with OpenSlide(SLIDE_FILE_PATH) as slide:
    thumbnail = slide.get_thumbnail((256, 256))
    print("get_thumbnail:", thumbnail)
    thumbnail.show()

In [ ]:
with OpenSlide(SLIDE_FILE_PATH) as slide:
    region = slide.read_region((0, 0), 4, (300, 300))
    print("read_region:", region)
    region.show()

In [ ]:
from openslide import OpenSlideCache

with OpenSlide(SLIDE_FILE_PATH) as slide:
    slide.set_cache(OpenSlideCache(1024))

## Deep Zoom Support

-   openslide object를 tile로 쪼개서 쉽게 접근할 수 있도록 한 것.


In [ ]:
from openslide import OpenSlide
from openslide.deepzoom import DeepZoomGenerator

with OpenSlide(SLIDE_FILE_PATH) as slide:
    deepzoom = DeepZoomGenerator(slide)  # (tile_size + 2 * overlap)이 2의 거듭제곱이 되도록
    print("level_count:", deepzoom.level_count)

    print("tile_count:", deepzoom.tile_count)

    print("level_tiles")
    for i, value in enumerate(deepzoom.level_tiles):
        print(f"\t{i:<2} {value}")

    print("level_dimensions")
    for i, value in enumerate(deepzoom.level_dimensions):
        print(f"\t{i:<2} {value}")

In [ ]:
with OpenSlide(SLIDE_FILE_PATH) as slide:
    deepzoom = DeepZoomGenerator(slide)
    print("get_dzi:", deepzoom.get_dzi("jpeg"))  # or 'png'

In [ ]:
with OpenSlide(SLIDE_FILE_PATH) as slide:
    deepzoom = DeepZoomGenerator(slide)
    tile = deepzoom.get_tile(15, (40, 40))
    print("get_tile:", tile)
    tile.show()

In [ ]:
with OpenSlide(SLIDE_FILE_PATH) as slide:
    deepzoom = DeepZoomGenerator(slide)
    coord = deepzoom.get_tile_coordinates(15, (40, 40))
    print("get_tile_coordinates:", coord)

    region = slide.read_region(*coord)
    print("read_region by coordinates:", region)
    region.show()

In [ ]:
with OpenSlide(SLIDE_FILE_PATH) as slide:
    deepzoom = DeepZoomGenerator(slide)
    # tile 1개 크기
    print("get_tile_dimensions:", deepzoom.get_tile_dimensions(15, (40, 40)))
    print("get_tile_dimensions:", deepzoom.get_tile_dimensions(8, (0, 0)))
    print("get_tile_dimensions:", deepzoom.get_tile_dimensions(4, (0, 0)))